This project aims to determine how the factors of a movie's production influence its eventual revenue take. It will focus on things that can be controlled during the actual producing of the movie, like budget and genre, and less on things that are out of the production crew's hands, like economic conditions or marketing success. It will not focus on small projects that cost very little to make and made very little at the box office. More movie theater work, less student or  small indie film. After data cleaning and transforming, results will be obtained by creating a simple linear regression model with revenue as the response variable, and seeing which of the predictor variables are most important and influential. The dataset used in this project can be found at https://www.kaggle.com/rounakbanik/the-movies-dataset. It was created by a data scientist named Rounak Banik and contains various datapoints on over 45,000 movies, with the data having been derived from movie information and review websites like TMDB and GroupLens. Let us start by importing the movies_metadata.csv data and taking a preliminary look at it.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("movies_metadata.csv")
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,6934,tt0419279,zu,Yesterday,"After falling ill, Yesterday learns that she i...",...,9/3/2004,0.0,96.0,"[{'iso_639_1': 'zu', 'name': 'isiZulu'}]",Released,NaN,Yesterday,False,8.3,6.0
1,False,NaN,94000000,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",http://www.theflowersofwarmovie.com/,76758,tt1410063,zh,金陵十三釵,A Westerner finds refuge with a group of women...,...,12/15/2011,95311434.0,145.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,NaN,The Flowers of War,False,7.1,193.0
2,False,"{'id': 392206, 'name': 'The Monkey King Collec...",82000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://themonkeyking.com/,119892,tt1717715,zh,西遊記之大鬧天宮,"Sun Wukong, (The Monkey King) is a monkey born...",...,1/9/2014,0.0,119.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,Havoc in Heaven,The Monkey King,False,4.8,52.0
3,False,"{'id': 96677, 'name': 'Red Cliff Collection', ...",80341000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",http://www.redclifffilm.com,12289,tt0425637,zh,Chi bi,"In the early third century, the land of Wu is ...",...,7/10/2008,127814609.0,150.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,The future will be decided.,Red Cliff,False,7.1,207.0
4,False,"{'id': 96677, 'name': 'Red Cliff Collection', ...",80000000,"[{'id': 10752, 'name': 'War'}, {'id': 28, 'nam...",http://www.redclifffilm.com,15384,tt1326972,zh,赤壁 2,"In 208 A.D., in the final days of the Han Dyna...",...,1/7/2009,121059225.0,136.0,"[{'iso_639_1': 'zh', 'name': '普通话'}, {'iso_639...",Released,Destiny Lies In The Wind,Red Cliff Part II,False,7.1,110.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  float64
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

We can see there are plenty of columns here. Before removing any variables, let us first create a few new variables to more easily utilize our dataset. These will largely take the form of transforming a object/string column into a boolean column.$\newline$ 
belongs_to_collection currently stores a string that corresponds to any series that movie is a part of. We will turn this into a boolean where True means the movie is a part of a series, and False means the movie is independent. $\newline$
original_language contains a two-letter abbreviation of the primary language spoken in the film. From this, original_english and original_chinese boolean variables will be created that is True when the movie's original_language is English (en) or Chinese (zh) respectively, and False when it is not. $\newline$
A series of genre boolean variables will be created to split apart the genres variable into multiple columns. From the genres in the dataset, we will use action_adventure, drama, thriller_horror, and comedy as our genres. It should be noted these are nonexclusive; a movie can contain True values in multiple genres. $\newline$
We may also need to transform the release_date column into something more workable, as it is currently a series of non-numeric strings. $\newline$
Let's start with belongs_to_collection. We can accomplish this change by creating a series using .map(), and assigning this series to the original column.

In [4]:
df["belongs_to_collection"]

0                                                      NaN
1                                                      NaN
2        {'id': 392206, 'name': 'The Monkey King Collec...
3        {'id': 96677, 'name': 'Red Cliff Collection', ...
4        {'id': 96677, 'name': 'Red Cliff Collection', ...
                               ...                        
45458                                                  NaN
45459                                                  NaN
45460                                                  NaN
45461                                                  NaN
45462                                                  NaN
Name: belongs_to_collection, Length: 45463, dtype: object

In [5]:
map = {np.nan:False}
sub = df["belongs_to_collection"].map(map)
map = {np.nan:True, False:False}
sub = sub.map(map)
df.loc[:,"belongs_to_collection"] = sub
df["belongs_to_collection"]

0        False
1        False
2         True
3         True
4         True
         ...  
45458    False
45459    False
45460    False
45461    False
45462    False
Name: belongs_to_collection, Length: 45463, dtype: bool

That's belongs_to_collection done, with previous NaN values becoming False and anything else becoming true. Pandas has also done us the courtesy of making the column a boolean! $\newline$
We'll make the language variables next. Since we need to do two languages, we'll create a function that will do the work. It will also use .map() to accomplish our task.

In [6]:
df["original_language"]

0         zu
1         zh
2         zh
3         zh
4         zh
        ... 
45458    NaN
45459    NaN
45460    NaN
45461    NaN
45462    NaN
Name: original_language, Length: 45463, dtype: object

In [7]:
def map_language(lang): 
    map = {str(lang):True}
    language = df["original_language"].map(map)
    map = {np.nan:False, True:True}
    language = language.map(map)
    return language

In [8]:
df["original_english"] = map_language("en")
df["original_english"][0:5]

0    False
1    False
2    False
3    False
4    False
Name: original_english, dtype: bool

In [9]:
df["original_english"][13535:13540]

13535    True
13536    True
13537    True
13538    True
13539    True
Name: original_english, dtype: bool

In [10]:
df["original_english"].describe()

count     45463
unique        2
top        True
freq      32269
Name: original_english, dtype: object

In [11]:
df["original_chinese"] = map_language("zh")
df["original_chinese"]

0        False
1         True
2         True
3         True
4         True
         ...  
45458    False
45459    False
45460    False
45461    False
45462    False
Name: original_chinese, Length: 45463, dtype: bool

In [12]:
df["original_chinese"].describe()

count     45463
unique        2
top       False
freq      45054
Name: original_chinese, dtype: object

Our language variables seem to be have made successfully, with False and True placed where appropriate. We can use describe() is used to get a basic idea of our languages; it seems like 32269 movies are English, and 45054 movies are not Chinese. For some reason describe() suggests the dtype is an object, but it seems like this is a consequence of how describe() works; there should be no worry about our columns not being booleans.$\newline$
Our final set of created columns are the genre columns: action_adventure, drama, thriller_horror, and comedy. We can use apply() with lambda x representing the value of each row, and Python's in operator to check for the presence of the genre.

In [13]:
df["genres"]

0                            [{'id': 18, 'name': 'Drama'}]
1        [{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...
2        [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
3        [{'id': 12, 'name': 'Adventure'}, {'id': 18, '...
4        [{'id': 10752, 'name': 'War'}, {'id': 28, 'nam...
                               ...                        
45458                  [{'id': 99, 'name': 'Documentary'}]
45459                        [{'id': 18, 'name': 'Drama'}]
45460    [{'id': 16, 'name': 'Animation'}, {'id': 99, '...
45461                                                   []
45462    [{'id': 28, 'name': 'Action'}, {'id': 10752, '...
Name: genres, Length: 45463, dtype: object

In [14]:
df["action_adventure"] = df["genres"].apply(lambda x: "Action" in x or "Adventure" in x)
df["action_adventure"]

0        False
1        False
2         True
3         True
4         True
         ...  
45458    False
45459    False
45460    False
45461    False
45462     True
Name: action_adventure, Length: 45463, dtype: bool

In [15]:
df["drama"] = df["genres"].apply(lambda x: "Drama" in x)
df["drama"]

0         True
1         True
2        False
3         True
4         True
         ...  
45458    False
45459     True
45460    False
45461    False
45462     True
Name: drama, Length: 45463, dtype: bool

In [16]:
df["thriller_horror"] = df["genres"].apply(lambda x: "Thriller" in x or "Horror" in x)
df["thriller_horror"]

0        False
1        False
2        False
3        False
4         True
         ...  
45458    False
45459    False
45460    False
45461    False
45462    False
Name: thriller_horror, Length: 45463, dtype: bool

In [17]:
df["comedy"] = df["genres"].apply(lambda x: "Comedy" in x)
df["comedy"][100:110]

100    False
101    False
102    False
103    False
104    False
105    False
106    False
107    False
108     True
109    False
Name: comedy, dtype: bool

The genre columns seem to have been created successfully, which True and False values present in boolean columns. $\newline$ 
Our last transformation will focus on release date. Let's create a release_year column from release_date. $\newline$

In [18]:
df["release_date"]

0          9/3/2004
1        12/15/2011
2          1/9/2014
3         7/10/2008
4          1/7/2009
            ...    
45458    1895-04-01
45459      1/1/2001
45460     3/12/2016
45461     1/27/2016
45462    1897-01-01
Name: release_date, Length: 45463, dtype: object

In [19]:
def get_year(date) :
    if ("-" in str(date)) :
        return str(date)[0:4]
    else :
        return str(date)[-4:]

In [20]:
df["release_year"] = df["release_date"].apply(get_year)
df["release_year"] = df["release_year"].astype("int32", errors = "ignore")
df["release_year"].describe()

count     45463
unique      136
top        2014
freq       1974
Name: release_year, dtype: object

In [21]:
df["release_year"]

0        2004
1        2011
2        2014
3        2008
4        2009
         ... 
45458    1895
45459    2001
45460    2016
45461    2016
45462    1897
Name: release_year, Length: 45463, dtype: object

We've now finished all our transformation columns. Let's use .info() again to remind us of what the overall dataset looks like. Remember, our goal is to eventually determine which of these columns are influential on the movie's revenue value via linear regression.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  bool   
 1   belongs_to_collection  45463 non-null  bool   
 2   budget                 45463 non-null  int64  
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  int64  
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  float64
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

Our dataset has a lot of variables, but some of them are not going to be useful in our analysis. For example, columns relating to review scores and voting are not useful since these values are only known after the movie is complete. The website column is entirely superfluous to predicting revenue. Things like the title and overview are very difficult to quantify, and would likely require intensive research into the movie industry, the use of language in titles, and potentially the psychology of word recognition. These are out of the scope of this project. $\newline$
Let's narrow our focus to variables that seem more likely to be influential, and only keep variables that apply to our analysis. We'll keep our created variables and drop the columns they were based on (belongs_to_collection has already been replaced). We will also keep revenue (Movie Revenue), adult (Whether or not the movie is an Adult film), runtime (length in minutes), budget (self explanatory), and release_year instead of release_date. release_year is kept largely for later subsetting, as using it in an analysis of revenue would ignore things like inflation and the overall growth of the industry.

In [23]:
keep = ["adult", "belongs_to_collection", "budget", "release_year", "revenue", "runtime", 
        "original_english", "original_chinese", "action_adventure", "drama", "thriller_horror", "comedy"]
sub = df.loc[:,keep]
sub

,adult,belongs_to_collection,budget,release_year,revenue,runtime,original_english,original_chinese,action_adventure,drama,thriller_horror,comedy
0,False,False,0,2004,0.0,96.0,False,False,False,True,False,False
1,False,False,94000000,2011,95311434.0,145.0,False,True,False,True,False,False
2,False,True,82000000,2014,0.0,119.0,False,True,True,False,False,False
3,False,True,80341000,2008,127814609.0,150.0,False,True,True,True,False,False
4,False,True,80000000,2009,121059225.0,136.0,False,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
45458,False,False,0,1895,0.0,1.0,False,False,False,False,False,False
45459,False,False,0,2001,0.0,0.0,False,False,False,True,False,False
45460,False,False,0,2016,0.0,76.0,False,False,False,False,False,False
45461,False,False,0,2016,0.0,84.0,False,False,False,False,False,False


We've done our column subsetting and have 12 columns left to work with. A brief glance at the data we have shows some problems in row data that will also need to be subsetted out. $\newline$
We'll subset budget and revenue to movies where these both exceed 500,000, and runtime to values above 30. This is admittedly arbitrary, but such a subset seems necessary, especially with numerous rows containing 0 or other low numbers for these values (potentially short films, student films, or simply missing data). We'll also subset release_year to movies released during or after 2000 to help ensure results are relevant to the present, before dropping release_year since it won't be a part of our linear regression analysis.$\newline$

In [24]:
sub = sub[sub["budget"] >= 500000]
sub = sub[sub["revenue"] >= 500000]
sub = sub[sub["runtime"] >= 30]
sub

,adult,belongs_to_collection,budget,release_year,revenue,runtime,original_english,original_chinese,action_adventure,drama,thriller_horror,comedy
1,False,False,94000000,2011,95311434.0,145.0,False,True,False,True,False,False
3,False,True,80341000,2008,127814609.0,150.0,False,True,True,True,False,False
4,False,True,80000000,2009,121059225.0,136.0,False,True,True,True,True,False
5,False,False,65000000,2015,121545703.0,127.0,False,True,True,True,False,False
6,False,False,65000000,2017,254212245.0,130.0,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
45034,False,False,7500000,1995,32392047.0,91.0,False,False,True,False,True,True
45045,False,False,1549000,2005,7096000.0,107.0,False,False,True,True,True,False
45046,False,True,900000,1992,20483423.0,95.0,False,False,True,False,True,False
45329,False,False,6000000,2010,3770657.0,108.0,False,False,False,True,False,False


In [25]:
sub.loc[:,"release_year"] = sub["release_year"].astype("int32")
sub = sub[sub["release_year"] >= 2010]
sub = sub.drop("release_year", axis=1)
sub

,adult,belongs_to_collection,budget,revenue,runtime,original_english,original_chinese,action_adventure,drama,thriller_horror,comedy
1,False,False,94000000,95311434.0,145.0,False,True,False,True,False,False
5,False,False,65000000,121545703.0,127.0,False,True,True,True,False,False
6,False,False,65000000,254212245.0,130.0,False,True,True,False,False,True
7,False,False,50000000,102205175.0,124.0,False,True,True,False,False,True
13,False,False,40000000,385284817.0,111.0,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
44676,False,False,3800000,16800000.0,115.0,False,False,False,True,False,False
45017,False,True,36000000,156844753.0,105.0,False,False,True,True,False,False
45019,False,False,29000000,33662874.0,131.0,False,False,True,True,False,False
45027,False,True,12902809,36000000.0,109.0,False,False,True,True,False,False


After all of our data cleaning and transforming and subsetting, we're left with 1450 rows of 11 columns to work with (revenue and 10 predictor variables). We'll need to convert our boolean True/False columns to numeric 1/0 columns for the linear modeling. We'll run a simple linear regression with revenue as the response variable against all the other variables, and make further decisions from there. We'll use a statistical package called Pingouin for our regression.

In [26]:
#If needed, remove hashtag below to install Pingouin
#! pip install pingouin

ERROR: Could not find a version that satisfies the requirement upgrade (from versions: none)
ERROR: No matching distribution found for upgrade


In [27]:
import pingouin as pg

In [28]:
sub = sub.astype(int)
sub

,adult,belongs_to_collection,budget,revenue,runtime,original_english,original_chinese,action_adventure,drama,thriller_horror,comedy
1,0,0,94000000,95311434,145,0,1,0,1,0,0
5,0,0,65000000,121545703,127,0,1,1,1,0,0
6,0,0,65000000,254212245,130,0,1,1,0,0,1
7,0,0,50000000,102205175,124,0,1,1,0,0,1
13,0,0,40000000,385284817,111,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
44676,0,0,3800000,16800000,115,0,0,0,1,0,0
45017,0,1,36000000,156844753,105,0,0,1,1,0,0
45019,0,0,29000000,33662874,131,0,0,1,1,0,0
45027,0,1,12902809,36000000,109,0,0,1,1,0,0


In [29]:
revenue = sub["revenue"]
variables = sub.drop("revenue", axis=1)
lm = pg.linear_regression(variables, revenue)
lm

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-7.481625e+07,350.886515,-2.132206e+05,0.000000e+00,0.656813,0.654668,-7.481694e+07,-7.481556e+07
1,belongs_to_collection,1.096642e+08,32.040104,3.422718e+06,0.000000e+00,0.656813,0.654668,1.096642e+08,1.096643e+08
2,budget,3.028417e+00,0.066540,4.551276e+01,5.143664e-281,0.656813,0.654668,2.897891e+00,3.158943e+00
3,runtime,5.324637e+05,40100.734666,1.327815e+01,4.838867e-38,0.656813,0.654668,4.538016e+05,6.111258e+05
4,original_english,6.228998e+06,256.197211,2.431329e+04,0.000000e+00,0.656813,0.654668,6.228496e+06,6.229501e+06
5,original_chinese,6.029274e+07,4.689337,1.285741e+07,0.000000e+00,0.656813,0.654668,6.029273e+07,6.029275e+07
6,action_adventure,-1.787632e+07,70.636992,-2.530731e+05,0.000000e+00,0.656813,0.654668,-1.787646e+07,-1.787618e+07
7,drama,-3.848926e+06,218.760189,-1.759427e+04,0.000000e+00,0.656813,0.654668,-3.849355e+06,-3.848497e+06
8,thriller_horror,-6.086223e+06,120.393194,-5.055288e+04,0.000000e+00,0.656813,0.654668,-6.086459e+06,-6.085987e+06
9,comedy,-7.825040e+05,117.349275,-6.668162e+03,0.000000e+00,0.656813,0.654668,-7.827342e+05,-7.822738e+05


C:\Users\Christopher Wang\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.0, the latest is 0.2.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


With the linear model complete, we're just about finished with coding and technical work for this project, all that's left is the (subjective) analysis and interpretation of the model.$\newline$
First, some general notes of the model as a whole. Our r2 (or $R^2$) and adj_r2 (or adjusted $R^2$) are .6568 and .6547, respectively. These are measures of how well the model is able to predict the values of the response variable, here revenue, in comparison to the actual observed values of revenue; so this model can predict about 65% of the variance of the revenue variable. (Adjusted $R^2$ is a modification to $R^2$ that is meant to penalize a model for using too many variables and overfitting to the sample instead of appropriately attempting to fit to a larger population. It being about the same as $R^2$ here is a good sign.) $\newline$
Next, a brief explanation to the important aspects of each row of output. The coef (coefficient) of a variable is a measure of how much that variable impacts revenue. A coefficient of 1 would mean for each unit the variable increases (or for the booleans, if the variable is True or 1), revenue also increases by 1. The e number means $10^{number}$. The pval (p value) of the variable is a measure of its significance, and is the result of an implicit hypothesis test that tests whether or not the "true population" coefficient of the variable is nonzero. Lower p-values, below $\alpha = .01$, are generally thought of as a good indication the variable's true coefficient is nonzero, and as such that variable is significant. The CI columns are a result of a 95% confidence interval attempting to estimate the true population coefficient value. $\newline$
With that, we can look at the values of each row and see what they can tell us. It is somewhat surprising to see that every single variable included has a very small p-value associated with it, with many of them being practically zero, indicating every single population coefficient for the variables is nonzero, and as such every single variable has a statistically significant impact on revenue. This can be double checked by observing no confidence interval includes zero. This means that any conclusions to be made on what factors are most influential on revenue will be subjective and will have to involve meta-thinking of the data. $\newline$
The only variable that doesn't appear is adult, which either means a film's status as an adult film doesn't impact its average revenue, or there were basically zero films that actually were adult left in our subset. $\newline$
Moving on to the genres, all the genres have negative coefficients associated with them, which indicates that movies fitting none of these genres are more likely to have higher revenues. The magnitudes of the coefficients also differ quite a bit, with comedy "decreasing" revenue the least, followed by drama and thriller_horror, and action_adventure "decreasing" revenue the most. $\newline$
The two language variables both have positive coefficients, with original_chinese having a e+07 vs original_english's e+06. Movies released in these two languages are more likely to have higher revenues. Looking internationally, it may be that movies produced for the Chinese and American/Western markets are more likely to have higher revenue than movies made in/for other markets. $\newline$
Runtime and budget both have positive coefficients as well. This could be for a number of reasons. Movies, particularly big blockbusters, are becoming longer and more expensive to produce, especially with the growing proliferation of superhero movies with expensive effects and star studded casts. In addition, films with higher runtimes and budgets are also likely to get more attention from the studio and have more marketing attached to them, which could contribute to higher revenues. $\newline$
Finally, belongs_to_collection has a positive coefficient, and quite a high one at that. This indicates movies that are part of a largely series are likely to have higher revenues. This seems reasonable, as a sequel can attract fans of the original film and new fans that haven't seen the previous movies. But it should also be taken with a grain of salt; while it is true the sequels in the dataset have higher average revenues, these sequels may have only been produced because of the original movie's successful revenue take in the first place. $\newline$ $\newline$

Of the usable variables within this movie dataset, including a movie's status as a part of a series, its budget and runtime, and its language and genre, practically all of them have a statistically significant impact on the movie's revenue. According to the model generated, highly-budgeted and long movies released in Chinese or English and without fitting into any of the included genres (or only fits into the comedy genre) will have higher average revenues than the "average" film.